# Setup

In [3]:
import numpy as np

import seaborn as sns
import pandas as pd
import os
import pprint
import ast
import re
import matplotlib.pyplot as plt
import dill
import requests
import xmltodict
import json

pp = pprint.PrettyPrinter(depth=6)

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

ALLOWED_METAL_NAMES =   {'Iron': 'FE+2', 'Cobalt': 'CO+2', 'Copper': 'CU+2', 'Manganese': 'MN+2', 'Molybdenum': 'CPD-8123', 'Nickel': 'NI+2', 'Zinc': 'ZN+2',
                        'Calcium': 'CA+2', 'Magnesium': 'MG+2', 'Sodium': 'NA+', 'Potassium': 'K+',
                        'Iron-sulfur \(4Fe-4S\)': 'CPD-7', 'Iron-sulfur \(2Fe-2S\)': 'CPD-6',
                         'Iron-sulfur \(4Fe-4S-S-AdoMet\)': 'CPD-7', 'Iron-sulfur \(3Fe-4S\)': '3FE-4S', 'Iron-oxo-sulfur \(4Fe-2O-2S\)': 'CPD-7',
                        'heme': 'Heme-b', 'Molybdate': 'CPD-3', 'heme B': 'Heme-b',
                         'L-SELENOCYSTEINE': 'L-SELENOCYSTEINE',
                        'Divalent metal cation': 'Any+2'}


ACCEPTED_OTHER_FEATURES = {'PYRIDOXAL_PHOSPHATE', 'THIAMINE-PYROPHOSPHATE', 'FMN', 'FAD'}

## Connect to api

In [5]:
password = input("Enter Password: ")

In [6]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password': password})

<Response [200]>

## api test

In [7]:
# example entry
# entity = 'PWY0-1356'
entity = 'CHEW-MONOMER'
req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{entity}&detail=high'

r = s.get(req_str)
if r.status_code != 200:
    print(entity, r.status_code)

o = xmltodict.parse(r.content)
pp.pprint(o['ptools-xml']['Protein'])

{'@ID': 'ECOLI:CHEW-MONOMER',
 '@detail': 'full',
 '@frameid': 'CHEW-MONOMER',
 '@orgid': 'ECOLI',
 'citation': [{'Publication': {'@ID': 'ECOLI:PUB-21463637',
                               '@detail': 'full',
                               '@frameid': 'PUB-21463637',
                               '@orgid': 'ECOLI',
                               'author': [{'#text': 'Underbakke ES',
                                           '@datatype': 'string'},
                                          {'#text': 'Zhu Y',
                                           '@datatype': 'string'},
                                          {'#text': 'Kiessling LL',
                                           '@datatype': 'string'}],
                               'pubmed-id': {'#text': '21463637',
                                             '@datatype': 'string'},
                               'source': {'#text': 'J Mol Biol 409(4);483-95',
                                          '@datatype': 'string'},
  

In [8]:
entity = 'Fermentation'

fn = 'get-class-all-instances'
req_str = f'https://websvc.biocyc.org/apixml?fn={fn}&id=ECOLI:{entity}&detail=none'

r = s.get(req_str)
if r.status_code != 200:
    print(entity, r.status_code)

o = xmltodict.parse(r.content)
pp.pprint(o['ptools-xml'])

{'@ptools-version': '27.0',
 '@xml:base': 'http://BioCyc.org/apixml?fn=get-class-all-instances%26id=ECOLI:Fermentation%26detail=NONE',
 'Pathway': [{'@frameid': 'PWY-5480',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5480'},
             {'@frameid': 'PWY0-1312',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY0-1312'},
             {'@frameid': 'PWY-5485',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5485'},
             {'@frameid': 'FERMENTATION-PWY',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:FERMENTATION-PWY'},
             {'@frameid': 'PWY-5437',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-5437'},
             {'@frameid': 'PWY-8274',
              '@orgid': 'ECOLI',
              '@resource': 'getxml?ECOLI:PWY-8274'}],
 'metadata': {'num_results': '6',
              'query': 'fn=get-class-all-instances&id=ECOLI:Fermenta

# Getting raw data

## Fetch EcoCyc pathway tree

In [9]:
def recursive_pathway_tree(current_node, parent_node, node_dict, level):


    if current_node in node_dict.keys():

        if level < node_dict[current_node]['level']:
            node_dict[current_node]['level'] = level

        node_dict[current_node]['parents'].append(parent_node)

        return

    else:

        req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{current_node}&detail=high'

        node_dict[current_node] = {'parents': [], 'children': [], 'level': level}

        if parent_node is not None:
            node_dict[current_node]['parents'].append(parent_node)

        r = s.get(req_str)
        if r.status_code != 200:
            print(current_node, r.status_code)
            return

        o = xmltodict.parse(r.content)


        subclasses = o['ptools-xml']['Pathway']['subclass'] if 'subclass' in o['ptools-xml']['Pathway'] else []
        if type(subclasses) is dict:
            subclasses = [subclasses]

        instances = o['ptools-xml']['Pathway']['instance'] if 'instance' in o['ptools-xml']['Pathway'] else []
        if type(instances) is dict:
            instances = [instances]

        pathways = subclasses + instances

        # print(f'{current_node}: {len(subclasses)}, {len(instances)}')

        for pathway in pathways:
            pathway_id = pathway['Pathway']['@frameid']

            node_dict[current_node]['children'].append(pathway_id)
            recursive_pathway_tree(pathway_id, current_node, node_dict, level+1)


    return


def get_pathway_ith_level_parents(cur_pathway_idx, pathway_matrix, name_list, level_vector, level=2, parent_dict=None):

    if parent_dict is None:
        parent_dict = {}

    cur_pathway_level = level_vector[cur_pathway_idx]

    if cur_pathway_level == level:
        parent_dict[name_list[cur_pathway_idx]] = cur_pathway_level

    parent_slice = pathway_matrix[:, cur_pathway_idx]
    parent_idxs = np.where(parent_slice != 0)[0]


    for idx in parent_idxs:

        _ = get_pathway_ith_level_parents(idx, pathway_matrix, name_list, level_vector, level, parent_dict)

    return parent_dict



In [10]:
entity = 'Pathways'

fn = 'get-class-direct-subs'
req_str = f'https://websvc.biocyc.org/apixml?fn={fn}&id=ECOLI:{entity}&detail=none'

r = s.get(req_str)
o = xmltodict.parse(r.content)['ptools-xml']['Pathway']
top_level_pathways = [pathway['@frameid'] for pathway in o]

pathway_node_dict = {}

for top_node in top_level_pathways:

    recursive_pathway_tree(top_node, None, pathway_node_dict, level=1)

ARG+POLYAMINE-SYN 404


In [11]:
pathway_df = pd.DataFrame(pathway_node_dict).T.reset_index(names='id')
pathway_df

,index,parents,children,level
0,Signaling-Pathways,[],"[PWY0-1559, PWY0-1495, PWY0-1518, PWY0-1490, P...",1
1,PWY0-1559,[Signaling-Pathways],[],2
2,PWY0-1495,[Signaling-Pathways],[],2
3,PWY0-1518,[Signaling-Pathways],[],2
4,PWY0-1490,[Signaling-Pathways],[],2
...,...,...,...,...
1159,GLUTATHIONESYN-PWY,[Reductants],[],4
1160,Butanediol-Biosynthesis,[Other-biosynthesis],[],3
1161,CYCLOPEPTIDES,[Other-biosynthesis],[],3
1162,6-HM-Dihydropterin-PP-Biosynthesis,[Other-biosynthesis],[PWY-6147],3


## Fetch protein monomer annotations

In [12]:
# get a set of all monomers with an associated uniprot id
proteins_df = pd.read_csv('reconstruction/ecoli/flat/proteins.tsv', sep='\t').loc[:, ["id", "common_name"]]

for column in ["enzyme_reaction", "cofactors", "metal_features", "other_features", "direct_annotations"]:
    proteins_df[column] = 0
    proteins_df[column] = proteins_df[column].astype(object)

proteins_df

,id,common_name,enzyme_reaction,cofactors,metal_features,other_features,direct_annotations
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,0,0,0,0,0
1,1-PFK-MONOMER,1-phosphofructokinase,0,0,0,0,0
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,0,0,0,0,0
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,0,0,0,0,0
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...",0,0,0,0,0
...,...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,0,0,0,0,0
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,0,0,0,0,0
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,0,0,0,0,0
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,0,0,0,0,0


In [13]:
for i in range(len(proteins_df.index)):
    if i % 100 == 0:
        print(i)

    protein = proteins_df.loc[i, 'id']
    proteins_df.at[i, 'other_features'] = set()
    proteins_df.at[i, 'metal_features'] = set()
    proteins_df.at[i, 'enzyme_reaction'] = set()
    proteins_df.at[i, 'cofactors'] = set()
    proteins_df.at[i, 'direct_annotations'] = set()

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{protein}&detail=high'



    r = s.get(req_str)
    if r.status_code != 200:
        print(protein, r.status_code)
        continue

    o = xmltodict.parse(r.content)['ptools-xml']



    metal_set = set()
    other_feature_set = set()
    if 'Protein' in o and 'has-feature' in o['Protein']:
        features = o['Protein']['has-feature']

        if type(features) is dict:
            features = [features]

        for feature in features:
            if 'parent' not in feature['Feature']:
                continue

            category = feature['Feature']['parent']['Feature']['@frameid']
            if category == 'Metal-Binding-Sites' and 'comment' in feature['Feature']:

                # Detect match to any of the allowed metal names and allowed cofactor names and add to list
                comment = feature['Feature']['comment']['#text']
                metal_set.add(comment)

                proteins_df.at[i, 'metal_features'] = list(metal_set)

            if category == 'Nucleotide-Phosphate-Binding-Regions' and 'attached-group' in feature['Feature'] and 'Compound' in feature['Feature']['attached-group']:
                attached_group = feature['Feature']['attached-group']['Compound']['@frameid']
                other_feature_set.add(attached_group)

            if category == 'N6-pyridoxal-phosphate-Lys-Modifications':
                other_feature_set.add('PYRIDOXAL_PHOSPHATE')

            if category == 'Protein-Segments' and 'comment' in feature['Feature'] and 'Thiamine' in feature['Feature']['comment']['#text']:
                other_feature_set.add('THIAMINE-PYROPHOSPHATE')

            if category == 'Selenocysteine-sites':
                print('found selenocysteine')
                metal_set.add('L-SELENOCYSTEINE')

            proteins_df.at[i, 'metal_features'] = list(metal_set)
            proteins_df.at[i, 'other_features'] = list(other_feature_set)

        if 'catalyzes' in o['Protein']:
            oc = o['Protein']['catalyzes']['Enzymatic-Reaction']

            if type(oc) is dict:
                oc = [oc]

            cofactor_set = set()
            enz_rxn_set = set()

            for enzrxn in oc:
                enz_id = enzrxn['@frameid']

                enz_rxn_set.add(enz_id)

                enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

                rz = s.get(enz_req_str)
                oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

                if "cofactor" in oe:
                    oe = oe['cofactor']

                    if type(oe) is dict:
                        oe = [oe]

                    for cofactor in oe:
                        cof = cofactor['Compound']['@frameid']
                        cofactor_set.add(cof)

            proteins_df.at[i, 'enzyme_reaction'] = enz_rxn_set
            proteins_df.at[i, 'cofactors'] = cofactor_set

    # get gene pathway annotation
    if 'Protein' in o and 'gene' in o['Protein']:

        gene = o['Protein']['gene']['Gene']['@frameid']
        gene_req_str = f'https://websvc.biocyc.org/apixml?fn=pathways-of-gene&id=ECOLI:{gene}&detail=none'

        rg = s.get(gene_req_str)
        og = xmltodict.parse(rg.content)['ptools-xml']

        if 'Pathway' in og:
            pathways = og['Pathway']

            if type(pathways) is dict:
                pathways = [pathways]

            for pathway in pathways:
                pathway_id = pathway['@frameid']
                proteins_df.at[i, 'direct_annotations'].add(pathway_id)




0
100
200
300
400
500
600
700
800
900
1000
1100
EG11708-MONOMER 404
1200
1300
1400
1500
1600
1700
found selenocysteine
found selenocysteine
found selenocysteine
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [14]:
proteins_df

,id,common_name,enzyme_reaction,cofactors,metal_features,other_features,direct_annotations
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,"{ENZRXN0-7991, ENZRXN0-8629, ENZRXN0-7995, ENZ...",{},[],[],"{PWY0-1319, PWY-5667}"
1,1-PFK-MONOMER,1-phosphofructokinase,{},{},[],[ATP],{PWY0-1314}
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,{2-DEHYDROPANTOATE-REDUCT-ENZRXN},{},[],[NADP],{PANTO-PWY}
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,{ENZRXN0-6250},{},[],[],{LEUSYN-PWY}
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...","{2-OCTAPRENYL-METHOXY-BENZOQ-METH-ENZRXN, ADOM...",{},[],[],"{PWY-6708, MENAQUINONESYN-PWY}"
...,...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,{},{},[],[ATP],{}
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,{},{},[],[],{}
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,{},{},"[zinc-coordinating residues, implicated in a s...",[],{}
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,{},{},[],[],{}


In [ ]:
# proteins_df.to_parquet('notebooks/fbagd/data/raw_protein_features.parquet', index=False)

## Reload protein feature table

In [15]:
# proteins_df = pd.read_parquet('notebooks/fbagd/data/raw_protein_features.parquet')
# proteins_df['metal_features'] = proteins_df['metal_features'].apply(ast.literal_eval)
# # for rows of proteins with where other_features is set, convert from string to set with literal_eval
# proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'] = \
#     proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'].apply(ast.literal_eval)

filter_protein_df = proteins_df.copy().loc[:, ['id', 'common_name', 'metal_features', 'other_features', 'enzyme_reaction', 'cofactors', 'direct_annotations']]
filter_protein_df

,id,common_name,metal_features,other_features,enzyme_reaction,cofactors,direct_annotations
0,1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER,1-acylglycerol-3-phosphate <i>O</i>-acyltransf...,[],[],"{ENZRXN0-7991, ENZRXN0-8629, ENZRXN0-7995, ENZ...",{},"{PWY0-1319, PWY-5667}"
1,1-PFK-MONOMER,1-phosphofructokinase,[],[ATP],{},{},{PWY0-1314}
2,2-DEHYDROPANTOATE-REDUCT-MONOMER,2-dehydropantoate 2-reductase,[],[NADP],{2-DEHYDROPANTOATE-REDUCT-ENZRXN},{},{PANTO-PWY}
3,2-ISOPROPYLMALATESYN-MONOMER,2-isopropylmalate synthase,[],[],{ENZRXN0-6250},{},{LEUSYN-PWY}
4,2-OCTAPRENYL-METHOXY-BENZOQ-METH-MONOMER,"bifunctional 2-octaprenyl-6-methoxy-1,4-benzoq...",[],[],"{2-OCTAPRENYL-METHOXY-BENZOQ-METH-ENZRXN, ADOM...",{},"{PWY-6708, MENAQUINONESYN-PWY}"
...,...,...,...,...,...,...,...
4415,YTFR-MONOMER,galactofuranose ABC transporter putative ATP b...,[],[ATP],{},{},{}
4416,YTFT-MONOMER,galactofuranose ABC transporter putative membr...,[],[],{},{},{}
4417,ZNUA-MONOMER,Zn<sup>2+</sup> ABC transporter periplasmic bi...,"[zinc-coordinating residues, implicated in a s...",[],{},{},{}
4418,ZNUB-MONOMER,Zn<sup>2+</sup> ABC transporter membrane subunit,[],[],{},{},{}


## Annotate complexation with EcoCyc data

In [16]:
complex_df = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions.tsv', sep='\t').loc[:, ['id', 'stoichiometry']]


removed_complexes = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions_removed.tsv', sep='\t')

# remove rows where id starts with '#'
complex_df = complex_df[~complex_df['id'].str.startswith('#')].reset_index(drop=True)

# remove rows of complex_df where id matches an id in removed_complexes
complex_df = complex_df[~complex_df['id'].isin(removed_complexes['id'])].reset_index(drop=True)
complex_df.stoichiometry = complex_df.stoichiometry.astype(object)


for i, stoich in enumerate(complex_df.loc[:, 'stoichiometry']):

    if type(stoich) is str and stoich[0] == '{':
        stoich = stoich.replace('null', '-1')
        stoich = ast.literal_eval(stoich)

        complex_df.at[i, 'stoichiometry'] = stoich

    else:
        complex_df.at[i, 'stoichiometry'] = {}


# for each row, find dict entry with positive value
for i in range(len(complex_df.index)):

    stoich = complex_df.loc[i, 'stoichiometry']

    for k,v in stoich.items():
        if v > 0:
            complex_df.at[i, 'id'] = k

complex_df

,id,stoichiometry
0,1-PFK,"{'1-PFK': 1, '1-PFK-MONOMER': -2}"
1,2OXOGLUTARATEDEH-CPLX,"{'2OXOGLUTARATEDEH-CPLX': 1, 'E1O': -1, 'E2O':..."
2,3-ISOPROPYLMALDEHYDROG-CPLX,"{'3-ISOPROPYLMALDEHYDROG-CPLX': 1, '3-ISOPROPY..."
3,3-ISOPROPYLMALISOM-CPLX,"{'3-ISOPROPYLMALISOM-CPLX': 1, 'LEUC-MONOMER':..."
4,3-METHYL-2-OXOBUT-OHCH3XFER-CPLX,"{'3-METHYL-2-OXOBUT-OHCH3XFER-CPLX': 1, '3-CH3..."
...,...,...
1063,CPLX0-8053,"{'CPLX0-8053': 1, 'EG10942-MONOMER': -1}"
1064,CPLX0-8253,"{'CPLX0-8253': 1, 'CSRC-RNA': -1, 'EG11447-MON..."
1065,SRP-CPLX,"{'SRP-CPLX': 1, 'EG10300-MONOMER': -1, 'FFS-RN..."
1066,CPLX0-7796APO,"{'CPLX0-7796APO': 1, 'PD04032': -2}"


In [17]:
complex_df["cofactors"] = 0
complex_df["cofactors"] = complex_df["cofactors"].astype(object)

complex_df["enzyme_reaction"] = 0
complex_df["enzyme_reaction"] = complex_df["enzyme_reaction"].astype(object)

for i in range(len(complex_df.index)):

    if i % 100 == 0:
        print(i)

    complex = complex_df.loc[i, 'id']

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{complex}&detail=low'

    r = s.get(req_str)
    if r.status_code != 200:
        print(complex, r.status_code)
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()
        continue

    o = xmltodict.parse(r.content)['ptools-xml']['Protein']


    # if enzyme
    if 'catalyzes' in o:
        o = o['catalyzes']['Enzymatic-Reaction']

        if type(o) is dict:
            o = [o]

        cofactor_set = set()
        enz_rxn_set = set()

        for enzrxn in o:
            enz_id = enzrxn['@frameid']

            enz_rxn_set.add(enz_id)

            enz_req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{enz_id}&detail=high'

            rz = s.get(enz_req_str)
            oe = xmltodict.parse(rz.content)['ptools-xml']['Enzymatic-Reaction']

            if "cofactor" in oe:
                oe = oe['cofactor']

                if type(oe) is dict:
                    oe = [oe]

                for cofactor in oe:
                    cof = cofactor['Compound']['@frameid']
                    cofactor_set.add(cof)

        complex_df.at[i, 'enzyme_reaction'] = enz_rxn_set
        complex_df.at[i, 'cofactors'] = cofactor_set

    else:
        complex_df.at[i, 'enzyme_reaction'] = set()
        complex_df.at[i, 'cofactors'] = set()

0
100
200
CPLX0-2423 404
300
CPLX0-3976 404
400
500
600
700
800
900
RECFOR-CPLX 404
1000
CPLX0-7796APO 404


## Get cofactor elemental composition and more

In [27]:
# get set of all cofactors
all_metal_cofactors = set(ALLOWED_METAL_NAMES.values())
all_other_cofactors = ACCEPTED_OTHER_FEATURES

filter_cofactor_df = pd.DataFrame(columns=['id', 'elemental_composition'])
filter_cofactor_df['id'] = list(all_metal_cofactors | all_other_cofactors)

filter_cofactor_df['elemental_composition'] = 0
filter_cofactor_df['elemental_composition'] = filter_cofactor_df['elemental_composition'].astype(object)

# for each cofactor, get elemental composition
for i in range(len(filter_cofactor_df.index)):

    compound = filter_cofactor_df.loc[i, 'id']
    atom_dict = {}

    url_name = compound.replace('+', '%2b')

    req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{url_name}&detail=full'

    r = s.get(req_str)
    if r.status_code != 200:
        print(compound, r.status_code)
        filter_cofactor_df.at[i, 'elemental_composition'] = {}
        continue

    o = xmltodict.parse(r.content)['ptools-xml']

    atoms = o['Compound']['cml']['molecule']['atomArray']['atom']
    if type(atoms) is dict:
        atoms = [atoms]

    for atom in atoms:
        element = atom['@elementType']

        # either add new element or add to existing element
        if element not in atom_dict:
            atom_dict[element] = 1
        else:
            atom_dict[element] += 1

    filter_cofactor_df.at[i, 'elemental_composition'] = atom_dict

Any+2 404


In [28]:
filter_cofactor_df

,id,elemental_composition
0,CPD-3,"{'MO': 1, 'O': 4}"
1,NA+,{'NA': 1}
2,MN+2,{'MN': 1}
3,FE+2,{'FE': 1}
4,FMN,"{'C': 17, 'N': 4, 'O': 9, 'P': 1}"
5,CPD-7,"{'FE': 4, 'S': 4}"
6,CU+2,{'CU': 1}
7,PYRIDOXAL_PHOSPHATE,"{'C': 8, 'N': 1, 'O': 6, 'P': 1}"
8,NI+2,{'NI': 1}
9,3FE-4S,"{'FE': 3, 'S': 4}"


## Save tables to avoid having to re-download

In [41]:
# save complex_df to csv in a way that preserves dicts
complex_df.to_csv('notebooks/cofactors/data/raw_complexes.csv', index=False)
proteins_df.to_csv('notebooks/cofactors/data/raw_proteins.csv', index=False)
filter_cofactor_df.to_csv('notebooks/cofactors/data/raw_cofactors.csv', index=False)
pathway_df.to_csv('notebooks/cofactors/data/raw_pathways.csv', index=False)

In [31]:
complex_df

,id,stoichiometry,cofactors,enzyme_reaction
0,1-PFK,"{'1-PFK': 1, '1-PFK-MONOMER': -2}",{MG+2},{1PFRUCTPHOSPHN-ENZRXN}
1,2OXOGLUTARATEDEH-CPLX,"{'2OXOGLUTARATEDEH-CPLX': 1, 'E1O': -1, 'E2O':...","{LIPOIC-ACID, THIAMINE-PYROPHOSPHATE, MG+2, FAD}",{2OXOGLUTARATEDEH-ENZRXN}
2,3-ISOPROPYLMALDEHYDROG-CPLX,"{'3-ISOPROPYLMALDEHYDROG-CPLX': 1, '3-ISOPROPY...","{MG+2, MN+2}",{3-ISOPROPYLMALDEHYDROG-ENZRXN}
3,3-ISOPROPYLMALISOM-CPLX,"{'3-ISOPROPYLMALISOM-CPLX': 1, 'LEUC-MONOMER':...",{CPD-7},{3-ISOPROPYLMALISOM-ENZRXN}
4,3-METHYL-2-OXOBUT-OHCH3XFER-CPLX,"{'3-METHYL-2-OXOBUT-OHCH3XFER-CPLX': 1, '3-CH3...",{MG+2},{3-METHYL-2-OXOBUT-OHCH3XFER-ENZRXN}
...,...,...,...,...
1063,CPLX0-8053,"{'CPLX0-8053': 1, 'EG10942-MONOMER': -1}",{},{}
1064,CPLX0-8253,"{'CPLX0-8253': 1, 'CSRC-RNA': -1, 'EG11447-MON...",{},{}
1065,SRP-CPLX,"{'SRP-CPLX': 1, 'EG10300-MONOMER': -1, 'FFS-RN...",{},{}
1066,CPLX0-7796APO,"{'CPLX0-7796APO': 1, 'PD04032': -2}",{},{}


## Reload data

In [15]:
parsed_complex_df = pd.read_csv('notebooks/cofactors/data/raw_complexes.csv', index_col=False)

# read stoichiometry, cofactors and enzyme_reaction as literal sets
for column in ['stoichiometry', 'cofactors', 'enzyme_reaction']:
    parsed_complex_df[column] = parsed_complex_df[column].apply(ast.literal_eval)

parsed_protein_df = pd.read_csv('notebooks/cofactors/data/raw_proteins.csv', index_col=False)

for column in ['cofactors', 'enzyme_reaction', 'metal_features', 'other_features', 'direct_annotations']:
    parsed_protein_df[column] = parsed_protein_df[column].apply(ast.literal_eval)


parsed_cofactor_df = pd.read_csv('notebooks/cofactors/data/raw_cofactors.csv', index_col=False)

for column in ['elemental_composition']:
    parsed_cofactor_df[column] = parsed_cofactor_df[column].apply(ast.literal_eval)

parsed_pathway_df = pd.read_csv('notebooks/cofactors/data/raw_pathways.csv', index_col=False)

for column in ['parents', 'children']:
    parsed_pathway_df[column] = parsed_pathway_df[column].apply(ast.literal_eval)

In [16]:
parsed_pathway_df

,index,parents,children,level
0,Signaling-Pathways,[],"[PWY0-1559, PWY0-1495, PWY0-1518, PWY0-1490, P...",1
1,PWY0-1559,[Signaling-Pathways],[],2
2,PWY0-1495,[Signaling-Pathways],[],2
3,PWY0-1518,[Signaling-Pathways],[],2
4,PWY0-1490,[Signaling-Pathways],[],2
...,...,...,...,...
1159,GLUTATHIONESYN-PWY,[Reductants],[],4
1160,Butanediol-Biosynthesis,[Other-biosynthesis],[],3
1161,CYCLOPEPTIDES,[Other-biosynthesis],[],3
1162,6-HM-Dihydropterin-PP-Biosynthesis,[Other-biosynthesis],[PWY-6147],3


# Data processing into final tables
## Processing of pathway data into matrices

In [12]:
name_list = list(pathway_node_dict.keys())

pathway_matrix = np.zeros((len(name_list), len(name_list)), dtype=np.int64)

level_vector = np.zeros(len(name_list), dtype=np.int64)

for i in range(len(name_list)):

    cur_pathway = name_list[i]
    level_vector[i] = pathway_node_dict[cur_pathway]['level']

    pathway_parents = pathway_node_dict[cur_pathway]['parents']
    pathway_children = pathway_node_dict[cur_pathway]['children']

    for parent in pathway_parents:
        j = name_list.index(parent)
        pathway_matrix[j, i] = 1

    for child in pathway_children:
        j = name_list.index(child)
        pathway_matrix[i, j] = 1

In [13]:
array_name_list = np.array(name_list)

In [14]:
pathway = 'PWY0-1321'
pathway_idx = name_list.index(pathway)

get_pathway_ith_level_parents(pathway_idx, pathway_matrix, name_list, level_vector, level=2)

{'Noncarbon-Nutrients': 2, 'Electron-Transfer': 2, 'Respiration': 2}

## Create new column for monomer component stoichiometry

In [ ]:
test_complex_df = pd.read_parquet('notebooks/fbagd/data/raw_complexes.parquet')
test_complex_df

# convert cofactors and enzyme_reaction to str
for col in ['stoichiometry', 'enzyme_reaction', 'cofactors']:
    test_complex_df[col] = test_complex_df[col].astype(str)

    # interpret as str literal
    test_complex_df[col] = test_complex_df[col].apply(lambda x: ast.literal_eval(x))

test_complex_df

complex_ids = complex_df['id'].tolist()
monomer_names = filter_protein_df['id'].tolist()

In [ ]:
def recursive_component_tree(current_component_name, complex_table, protein_table,
                             current_multiplier=1, component_list=None, parent=None, return_cofactors=False):
    """
    Recursively find all downstream components of a given complex.
    """

    complex_names = complex_table['id'].tolist()
    monomer_names = protein_table['id'].tolist()


    my_children = {}

    if component_list is None:
        component_list = []


    if current_component_name in complex_names:


        cplx_idx = complex_table.index[complex_table['id'] == current_component_name][0]
        stoichiometry = complex_table.at[cplx_idx, 'stoichiometry']

        direct_children = {k: abs(v) for k, v in stoichiometry.items() if v < 0}

        for component_name, coefficient in stoichiometry.items():

            if coefficient < 0 and component_name != current_component_name:

                child_multiplier = abs(coefficient * current_multiplier)

                new_child = recursive_component_tree(component_name, complex_table, protein_table,
                                                     child_multiplier, component_list, current_component_name, return_cofactors)

                my_children = my_children | new_child


            elif coefficient > 0 and component_name == current_component_name:
                continue

            else:
                raise ValueError(f"key {component_name} and value {coefficient} for complex {component_name} not processed properly.")

        component_list.append({'name': current_component_name,'parent': parent, 'children': direct_children,
                               'multiplier': int(current_multiplier), })


    elif current_component_name in monomer_names:

        # TODO check if enzrxn
        if return_cofactors:
            protein_idx = protein_table.index[protein_table['id'] == current_component_name][0]

            protein_metals = protein_table.at[protein_idx, 'metal_features_processed']
            protein_other = protein_table.at[protein_idx, 'other_features_processed']

            table_cofactors = protein_metals | protein_other

            if len(table_cofactors) > 0:
                # TODO Add apo protein to component list
                my_children = {}

                for cofactor, cofactor_coefficient in table_cofactors.items():
                    if table_cofactors[cofactor] !=  None:
                        my_children[cofactor] = cofactor_coefficient
                        component_list.append({'parent': current_component_name,
                                               'name': cofactor,
                                               'multiplier': abs(int(current_multiplier * cofactor_coefficient)),
                                               'children': None})

            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': my_children})

        else:
            my_children = None
            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': None})



    else:
        print(f"component {current_component_name} not found in complex or protein tables")

        return {}


    if parent is None:
        return {current_component_name: my_children}, component_list
    else:
        return {current_component_name: my_children}


In [ ]:
complex_tree_structure, nodes = recursive_component_tree('CPLX0-8167', complex_df, filter_protein_df)
pp.pprint(nodes)

In [ ]:
complex_df['monomer_component_stoichiometry'] = 0
complex_df['monomer_component_stoichiometry'] = complex_df['monomer_component_stoichiometry'].astype(object)

for i in range(len(complex_df.index)):
    complex_name = complex_df.loc[i, 'id']
    complex_tree_structure, nodes = recursive_component_tree(complex_name, complex_df, filter_protein_df)

    monomer_components = {node['name']: node['multiplier'] for node in nodes if node['children'] is None}

    complex_df.at[i, 'monomer_component_stoichiometry'] = monomer_components

In [ ]:
filter_complex_df = complex_df.loc[:, ["id", "monomer_component_stoichiometry", "cofactors"]]
filter_complex_df

In [ ]:
# filter_complex_df.to_parquet('notebooks/fbagd/data/processed_complexes.parquet', index=False)

In [ ]:
# save as parquet
# filter_cofactor_df.to_parquet('notebooks/fbagd/data/processed_cofactors.parquet', index=False)

Seems neglible. Let's just use the protein features.

## Process raw EcoCyc annotations into standard EcoCyc names

In [ ]:
# proteins_df = pd.read_parquet('notebooks/fbagd/data/raw_protein_features.parquet')
# proteins_df['metal_features'] = proteins_df['metal_features'].apply(ast.literal_eval)
# # for rows of proteins with where other_features is set, convert from string to set with literal_eval
# proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'] = \
#     proteins_df.loc[proteins_df['other_features'].str.startswith('['), 'other_features'].apply(ast.literal_eval)

filter_protein_df = proteins_df.copy().loc[:, ['id', 'common_name', 'metal_features', 'other_features', 'enzyme_reaction', 'cofactors', 'direct_annotations']]
filter_protein_df

In [ ]:

# remove all \ characters from keys in ALLOWED_METAL_NAMES
NON_REGEX_METAL = {key.replace('\\', ''): value for key, value in ALLOWED_METAL_NAMES.items()}

filter_protein_df['metal_features_processed'] = 0
filter_protein_df['metal_features_processed'] = filter_protein_df['metal_features_processed'].astype(object)

metal_pattern = '|'.join(ALLOWED_METAL_NAMES.keys())
metal_regex = re.compile(f'(({metal_pattern})(\s\d[\.,;]|[\.,;]|\s\())')


for i in range(len(filter_protein_df.index)):

    metal_binding = filter_protein_df.loc[i, 'metal_features']

    metal_count_dict = {}
    existing_matches = set()

    for feature in metal_binding:
        matches = metal_regex.search(feature)
        if matches:
            metal = matches.group(0)[:-1]

            # eliminate duplicates
            if metal not in existing_matches:

                existing_matches.add(metal)

                if 'heme' in feature:
                    metal = metal.replace('Iron', 'heme')

                # check if last char of metal is a number, then crop
                if metal[-1].isdigit():
                    metal = metal[:-2]

                metal = metal.strip()

                # replace metal name with allowed metal name
                metal = NON_REGEX_METAL[metal]

                if metal in metal_count_dict:
                    metal_count_dict[metal] += 1
                else:
                    metal_count_dict[metal] = 1

        else:
            print(f'No match for {feature} in {filter_protein_df.loc[i, "id"]}')




    filter_protein_df.at[i, 'metal_features_processed'] = metal_count_dict

In [ ]:
filter_protein_df = filter_protein_df.drop(columns=['metal_features'])
filter_protein_df

In [ ]:
filter_protein_df['other_features_processed'] = 0
filter_protein_df['other_features_processed'] = filter_protein_df['other_features_processed'].astype(object)

for i in range(len(filter_protein_df.index)):

    other_features = filter_protein_df.loc[i, 'other_features']

    other_feature_count_dict = {}
    existing_matches = set()

    for feature in other_features:

        # eliminate duplicates
        if feature not in existing_matches:

            existing_matches.add(feature)

            if feature in ACCEPTED_OTHER_FEATURES:
                if feature in other_feature_count_dict:
                    other_feature_count_dict[feature] += 1
                else:
                    other_feature_count_dict[feature] = 1

    filter_protein_df.at[i, 'other_features_processed'] = other_feature_count_dict

In [ ]:
filter_protein_df = filter_protein_df.drop(columns=['other_features'])

filter_protein_df

## Get monomer composition of complexes

In [ ]:
test_complex_df = pd.read_parquet('notebooks/fbagd/data/raw_complexes.parquet')
test_complex_df

# convert cofactors and enzyme_reaction to str
for col in ['stoichiometry', 'enzyme_reaction', 'cofactors']:
    test_complex_df[col] = test_complex_df[col].astype(str)

    # interpret as str literal
    test_complex_df[col] = test_complex_df[col].apply(lambda x: ast.literal_eval(x))

test_complex_df

complex_ids = complex_df['id'].tolist()
monomer_names = filter_protein_df['id'].tolist()

In [ ]:
def recursive_component_tree(current_component_name, complex_table, protein_table,
                             current_multiplier=1, component_list=None, parent=None, return_cofactors=False):
    """
    Recursively find all downstream components of a given complex.
    """

    complex_names = complex_table['id'].tolist()
    monomer_names = protein_table['id'].tolist()


    my_children = {}

    if component_list is None:
        component_list = []


    if current_component_name in complex_names:


        cplx_idx = complex_table.index[complex_table['id'] == current_component_name][0]
        stoichiometry = complex_table.at[cplx_idx, 'stoichiometry']

        direct_children = {k: abs(v) for k, v in stoichiometry.items() if v < 0}

        for component_name, coefficient in stoichiometry.items():

            if coefficient < 0 and component_name != current_component_name:

                child_multiplier = abs(coefficient * current_multiplier)

                new_child = recursive_component_tree(component_name, complex_table, protein_table,
                                                     child_multiplier, component_list, current_component_name, return_cofactors)

                my_children = my_children | new_child


            elif coefficient > 0 and component_name == current_component_name:
                continue

            else:
                raise ValueError(f"key {component_name} and value {coefficient} for complex {component_name} not processed properly.")

        component_list.append({'name': current_component_name,'parent': parent, 'children': direct_children,
                               'multiplier': int(current_multiplier), })


    elif current_component_name in monomer_names:

        # TODO check if enzrxn
        if return_cofactors:
            protein_idx = protein_table.index[protein_table['id'] == current_component_name][0]

            protein_metals = protein_table.at[protein_idx, 'metal_features_processed']
            protein_other = protein_table.at[protein_idx, 'other_features_processed']

            table_cofactors = protein_metals | protein_other

            if len(table_cofactors) > 0:
                # TODO Add apo protein to component list
                my_children = {}

                for cofactor, cofactor_coefficient in table_cofactors.items():
                    if table_cofactors[cofactor] !=  None:
                        my_children[cofactor] = cofactor_coefficient
                        component_list.append({'parent': current_component_name,
                                               'name': cofactor,
                                               'multiplier': abs(int(current_multiplier * cofactor_coefficient)),
                                               'children': None})

            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': my_children})

        else:
            my_children = None
            component_list.append({'parent': parent, 'name': current_component_name, 'multiplier': current_multiplier, 'children': None})



    else:
        print(f"component {current_component_name} not found in complex or protein tables")

        return {}


    if parent is None:
        return {current_component_name: my_children}, component_list
    else:
        return {current_component_name: my_children}


In [ ]:
complex_tree_structure, nodes = recursive_component_tree('CPLX0-8167', complex_df, filter_protein_df)
pp.pprint(nodes)

In [ ]:
complex_df['monomer_component_stoichiometry'] = 0
complex_df['monomer_component_stoichiometry'] = complex_df['monomer_component_stoichiometry'].astype(object)

for i in range(len(complex_df.index)):
    complex_name = complex_df.loc[i, 'id']
    complex_tree_structure, nodes = recursive_component_tree(complex_name, complex_df, filter_protein_df)

    monomer_components = {node['name']: node['multiplier'] for node in nodes if node['children'] is None}

    complex_df.at[i, 'monomer_component_stoichiometry'] = monomer_components

In [ ]:
filter_complex_df = complex_df.loc[:, ["id", "monomer_component_stoichiometry", "cofactors"]]
filter_complex_df

## Create tree matrix (also for Julia)

In [ ]:
# save names
complex_ids = list(filter_complex_df['id'])
protein_ids = list(filter_protein_df['id'])
cofactor_ids = list(filter_cofactor_df['id'])


name_idx = complex_ids + protein_ids + cofactor_ids
tree_matrix = np.zeros([len(complex_ids) + len(protein_ids) + len(cofactor_ids), len(complex_ids) + len(protein_ids) + len(cofactor_ids)], dtype=np.int64)

for i in range(len(filter_complex_df)):
    name = filter_complex_df.at[i, 'id']
    tree_structure, nodes = recursive_component_tree(name, complex_df, filter_protein_df, return_cofactors=True)

    for node in nodes:
        node_name = node['name']
        node_children = node['children']

        if node_children != None:
            for child_name, child_coefficient in node_children.items():
                if child_name in name_idx:
                        tree_matrix[name_idx.index(node_name), name_idx.index(child_name)] = child_coefficient

# Create matrices to get cofactor counts

In [ ]:
filter_protein_df

In [ ]:
# create protein name to index mapping
protein_name_to_index = {}
for i in range(len(filter_protein_df.index)):
    protein_name_to_index[filter_protein_df.at[i, 'id']] = i

# C matrix: complexes x proteins
C = np.zeros((len(filter_complex_df.index), len(filter_protein_df.index)))

for i in range(len(filter_complex_df.index)):

    complex_components = filter_complex_df.loc[i, 'monomer_component_stoichiometry']

    # TODO consider cofactors
    # complex_cofactors = filter_complex_df.loc[i, 'cofactors']


    for component_name, component_count in complex_components.items():
        if component_count is not None:             # side effect of parquet
            # get index of component in filter_protein_df
            component_index = protein_name_to_index[component_name]

            if filter_complex_df.at[i, 'id'] == 'APORNAP-CPLX':
                print(f'component_name: {component_name}, component_count: {component_count}, component_index: {component_index}')

            C[i, component_index] = component_count

# append an identity matrix to C
C = np.concatenate((C, np.identity(len(filter_protein_df.index))), axis=0)

C_names = list(filter_complex_df['id']) + list(filter_protein_df['id'])

In [ ]:
filter_complex_df.at[99, 'id']

In [ ]:
# create cofactor name to index mapping
cofactor_name_to_index = {}
for i in range(len(filter_cofactor_df.index)):
    cofactor_name_to_index[filter_cofactor_df.at[i, 'id']] = i

cofactor_ids = list(filter_cofactor_df['id'])

# P matrix: proteins x cofactors
P = np.zeros((len(filter_protein_df.index), len(filter_cofactor_df.index)))

for i in range(len(filter_protein_df.index)):
    protein_metals = filter_protein_df.loc[i, 'metal_features_processed']
    protein_other = filter_protein_df.loc[i, 'other_features_processed']

    for metal, count in protein_metals.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[metal]
            P[i, cofactor_index] = count

    for other, count in protein_other.items():
        if count is not None:             # side effect of parquet
            cofactor_index = cofactor_name_to_index[other]
            P[i, cofactor_index] = count


In [ ]:
P

In [ ]:
# create list of unique elements
unique_elements = set()
for i in range(len(filter_cofactor_df.index)):
    cofactor = filter_cofactor_df.at[i, 'elemental_composition']
    unique_elements.update(cofactor.keys())

unique_elements = list(unique_elements)

# create E matrix: cofactors x elements
E = np.zeros((len(filter_cofactor_df.index), len(unique_elements)))

for i in range(len(filter_cofactor_df.index)):
    cofactor = filter_cofactor_df.at[i, 'elemental_composition']

    for element, count in cofactor.items():
        if count is not None:             # side effect of parquet
            element_index = unique_elements.index(element)
            E[i, element_index] = count


element_ids = unique_elements

In [ ]:
unique_elements

In [ ]:
C_to_E = C @ P @ E

In [ ]:
C_names[1577]

# Now ... add the counts >:o

In [ ]:
time = '50'
date = '2023-06-09'
experiment = 'fba-redux'
entry = f'{experiment}_{time}_{date}'
folder = f'out/fbagd/{entry}/'

In [ ]:
output = np.load(folder + 'output.npy',allow_pickle='TRUE').item()
# output = np.load(r"out/geneRxnVerifData/output_glc.npy", allow_pickle=True, encoding='ASCII').tolist()
output = output['agents']['0']
fba = output['listeners']['fba_results']
mass = output['listeners']['mass']
bulk = pd.DataFrame(output['bulk'])

In [ ]:
bulk

In [ ]:
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux']
stoichiometry = metabolism.stoichiometry


In [ ]:
initial_state = json.load(open('data/wcecoli_t0.json'))

bulk_ids = [item[0] for item in initial_state['bulk']]

bulk.columns = bulk_ids

In [ ]:
ecocyc_to_wcm_map = {}

# combined complex and protein names
complex_protein_names = list(filter_protein_df['id']) + list(filter_complex_df['id'])

for name in complex_protein_names:

    # find complex name in bulk_ids
    found = False

    try:
        idx = bulk_ids.index(name+'[c]')
        ecocyc_to_wcm_map[name] = name+'[c]'
        found = True
        # print(f'found {complex_name} at {idx}')

    except ValueError:
        # delete key
        found = False


    if found == False:

        for id in bulk_ids:
            if name+'[' in id and id.startswith(name) and bulk.loc[:, id].sum() > 0:
                #print(f'found {name} in {id} with nonzero count')
                ecocyc_to_wcm_map[name] = id
                found = True
                break           # ensures preferring nonzero counts

            elif name+'[' in id and id.startswith(name):
                # print(f'found {name} in {id} with zero count')
                ecocyc_to_wcm_map[name] = id
                found = True

    if found == False:
        ecocyc_to_wcm_map[name] = '--TRANS-ACENAPHTHENE-12-DIOL[j]' # should be none
        print(f'could not find {name}')


In [ ]:
complex_wcm_names = [ecocyc_to_wcm_map[name] for name in C_names]

counts = bulk.loc[0, complex_wcm_names]

# Finally ... add the counts >:o

In [ ]:
factored_cofactor_elements = np.array(counts).reshape(-1,1) * C @ P @ E

In [ ]:
factored_cofactor_elements

In [ ]:
{unique_elements[i]: factored_cofactor_elements.sum(axis=0)[i] for i in range(len(unique_elements))}

In [ ]:
cidx = C_names.index('APORNAP-CPLX')
{unique_elements[i]: factored_cofactor_elements[cidx,i] for i in range(len(unique_elements)) if factored_cofactor_elements[cidx,i] > 0}

# Saving outputs to files compatible with Julia

In [ ]:
complex_ids = list(filter_complex_df['id'])
protein_ids = list(filter_protein_df['id'])
cofactor_ids = list(filter_cofactor_df['id'])


# save C, P and E to julia-compatible formats
np.savetxt('notebooks/fbagd/data/C_matrix.csv', C.astype(np.int64), delimiter=',', fmt='%i')
np.savetxt('notebooks/fbagd/data/P_matrix.csv', P.astype(np.int64), delimiter=',', fmt='%i')
np.savetxt('notebooks/fbagd/data/E_matrix.csv', E.astype(np.int64), delimiter=',', fmt='%i')



# write all names to single file with each list on a new line
with open('notebooks/fbagd/data/complex_ids.txt', 'w') as f:
    f.write('\n'.join(complex_ids))
with open('notebooks/fbagd/data/protein_ids.txt', 'w') as f:
    f.write('\n'.join(protein_ids))
with open('notebooks/fbagd/data/cofactor_ids.txt', 'w') as f:
    f.write('\n'.join(cofactor_ids))
with open('notebooks/fbagd/data/element_ids.txt', 'w') as f:
    f.write('\n'.join(element_ids))

# write all names to single file with each list on a new line
with open('notebooks/fbagd/data/complex_ids.txt', 'w') as f:
    f.write('\n'.join(complex_ids))
with open('notebooks/fbagd/data/protein_ids.txt', 'w') as f:
    f.write('\n'.join(protein_ids))
with open('notebooks/fbagd/data/cofactor_ids.txt', 'w') as f:
    f.write('\n'.join(cofactor_ids))


# save counts of proteins and complexes
np.savetxt('notebooks/fbagd/data/counts.csv', np.array(counts, dtype=np.int64), delimiter=',', fmt='%i')

# save tree_matrix
np.savetxt('notebooks/fbagd/data/tree_matrix.csv', tree_matrix, delimiter=',', fmt='%i')



In [ ]:
filter_complex_df

In [ ]:
for name in protein_ids:
    if 'CPLX' in name:
        print(name)

In [ ]:
counts[ecocyc_to_wcm_map["NAP-CPLX"]]

In [ ]:
ecocyc_to_wcm_map["NAP-CPLX"]

In [ ]:
"^"+'NAP-CPLX'+"[" in 'NAP-CPLX['

In [ ]:
protein_ids[1]

## go annotation for proteins

In [ ]:

# if 'Protein' in o and 'has-go-term' in o['Protein']:
#     go_terms = o['Protein']['has-go-term']
#
#     annotation_set = set()
#
#     if type(go_terms) is dict:
#         go_terms = [go_terms]
#
#     for go_term in go_terms:
#         # example entry
#         entity = go_term['GO-Term']['@frameid']
#         entity = entity.replace(':', '%3A')
#         req_str = f'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{entity}'
#
#         r = s.get(req_str)
#         if r.status_code != 200:
#             print(entity, r.status_code)
#
#         o = json.loads(r.content)
#         if len(o['results']) != 1:
#             print('error', entity, len(o['results']))
#             continue
#         elif o['results'][0]['aspect'] == 'biological_process':
#             annotation_set.add(o['results'][0]['name'])
#
#             o['ptools-xml']['Protein']['gene']['Gene']['@frameid']
#
#     proteins_df.at[i, 'direct_annotations'] = annotation_set